In [ ]:
# Lectura de datos desde archivo

In [ ]:
import numpy as np
import random

data = np.loadtxt('/content/drive/MyDrive/UGR/IC/Practica2/qap.datos/tai256c.dat', skiprows=1)

# Separación en matrices de flujos y distancias

In [ ]:
flow = np.int32(data[:256])
distances = np.int32(data[256:])

# Configuración de la semilla para reproducibilidad

In [ ]:
seed = 2024
random.seed(seed)
np.random.seed(seed)

# Generación de población inicial de 2000 individuos


In [ ]:
population = np.array([np.random.permutation(np.arange(256)) for _ in range(2000)])

# Función de fitness eficiente usando operaciones matriciales

In [ ]:
def fitness_pop(population, flow, distances):
    return np.sum(flow[np.newaxis, :, :] * distances[population[:,:,np.newaxis], population[:, np.newaxis, :]], axis=(1,2))


# Selección por torneo

In [ ]:
def selection_tournament(population, fitness_vals, k=3):
    selected_indices = np.array([random.choices(np.arange(len(fitness_vals)), k=k, weights=1/fitness_vals)[0] for _ in range(len(population))])
    return population[selected_indices]

# Cruce de orden (Order Crossover, OX)

In [ ]:
def crossover_order(parent1, parent2):
    n = len(parent1)
    start, end = sorted(random.sample(range(n), 2))
    child = -np.ones(n, dtype=int)
    child[start:end+1] = parent1[start:end+1]

    position = 0
    for gene in parent2:
        if gene not in child:
            while child[position] != -1:
                position += 1
            child[position] = gene
    return child

def crossover_population(population, crossover_prob=0.9):
    new_population = []
    for _ in range(len(population) // 2):
        parent1, parent2 = population[random.sample(range(len(population)), 2)]
        if random.random() < crossover_prob:
            child1 = crossover_order(parent1, parent2)
            child2 = crossover_order(parent2, parent1)
        else:
            child1, child2 = parent1.copy(), parent2.copy()
        new_population.extend([child1, child2])
    return np.array(new_population)

# Mutación por intercambio (Swap Mutation)

In [ ]:
def mutate_population(population, mutation_prob=0.3):
    for individual in population:
        if random.random() < mutation_prob:
            i, j = random.sample(range(len(individual)), 2)
            individual[i], individual[j] = individual[j], individual[i]
    return population

# Búsqueda local (2-opt limitada)

In [ ]:
def local_search_2opt_limited(permutation, flow, distances, max_iter=20):
    n = len(permutation)
    best_cost = calculate_cost(permutation, flow, distances)
    for _ in range(max_iter):
        i, j = sorted(random.sample(range(n), 2))
        new_perm = permutation.copy()
        new_perm[i:j+1] = list(reversed(new_perm[i:j+1]))
        new_cost = calculate_cost(new_perm, flow, distances)
        if new_cost < best_cost:
            permutation = new_perm
            best_cost = new_cost
    return permutation

def calculate_cost(permutation, flow, distances):
    indexed_distances = distances[permutation][:, permutation]
    cost = np.sum(flow * indexed_distances)
    return cost

# Variante Lamarckiana: aplicar la búsqueda local y actualizar permanentemente la población


In [ ]:
def apply_lamarckian_search(population, flow, distances, max_iter=20):
    # Mejorar cada individuo y actualizar permanentemente la población
    for i in range(len(population)):
        population[i] = local_search_2opt_limited(population[i], flow, distances, max_iter)
    return population

# Ciclo evolutivo

In [ ]:
generations = 200
crossover_prob = 0.9
mutation_prob = 0.3

best_solution = None
best_fitness = float('inf')

fitness_vals = fitness_pop(population, flow, distances)

for generation in range(generations):
    # Aplicar la variante Lamarckiana: mejora de la población de forma permanente
    population = apply_lamarckian_search(population, flow, distances)

    # Selección
    selected_population = selection_tournament(population, fitness_vals)

    # Cruce
    offspring = crossover_population(selected_population, crossover_prob)

    # Mutación
    population = mutate_population(offspring, mutation_prob)

    # Recalcular fitness
    fitness_vals = fitness_pop(population, flow, distances)

    # Guardar el mejor individuo
    gen_best_fitness = fitness_vals.min()
    if gen_best_fitness < best_fitness:
        best_fitness = gen_best_fitness
        best_solution = population[np.argmin(fitness_vals)]

    print(f"Generación {generation + 1}: Mejor Fitness = {best_fitness}")
    if best_fitness <= 44759294:
        print("Objetivo alcanzado, deteniendo el proceso.")
        break

print("Mejor solución encontrada:", best_solution)
print("Costo asociado:", best_fitness)


Generación 1: Mejor Fitness = 50066808
Generación 2: Mejor Fitness = 49998040
Generación 3: Mejor Fitness = 49998040
Generación 4: Mejor Fitness = 49678946
Generación 5: Mejor Fitness = 49643154
Generación 6: Mejor Fitness = 49643154
Generación 7: Mejor Fitness = 49363856
Generación 8: Mejor Fitness = 49363856
Generación 9: Mejor Fitness = 49363856
Generación 10: Mejor Fitness = 49363856
Generación 11: Mejor Fitness = 49363856
Generación 12: Mejor Fitness = 49363856
Generación 13: Mejor Fitness = 49363856
Generación 14: Mejor Fitness = 49363856
Generación 15: Mejor Fitness = 49363856
Generación 16: Mejor Fitness = 49363856
Generación 17: Mejor Fitness = 49363856
Generación 18: Mejor Fitness = 49363856
Generación 19: Mejor Fitness = 49363856
Generación 20: Mejor Fitness = 49363856
Generación 21: Mejor Fitness = 49363856
Generación 22: Mejor Fitness = 49363856
Generación 23: Mejor Fitness = 49363856
Generación 24: Mejor Fitness = 49363856
Generación 25: Mejor Fitness = 49363856
Generació

# Versión mejorada

In [ ]:
import numpy as np
import random
from joblib import Parallel, delayed

# Lectura de datos desde archivo
data = np.loadtxt('/content/drive/MyDrive/UGR/IC/Practica2/qap.datos/tai256c.dat', skiprows=1)

# Separación en matrices de flujos y distancias
flow = np.int32(data[:256])
distances = np.int32(data[256:])

# Configuración de la semilla para reproducibilidad
seed = 2024
random.seed(seed)
np.random.seed(seed)

# Generación de población inicial de 2000 individuos
population = np.array([np.random.permutation(np.arange(256)) for _ in range(2000)])

# Función de fitness eficiente usando operaciones matriciales
def fitness_pop(population, flow, distances):
    return np.sum(flow[np.newaxis, :, :] * distances[population[:,:,np.newaxis], population[:, np.newaxis, :]], axis=(1,2))

# Selección por torneo con elitismo
def selection_tournament_with_elitism(population, fitness_vals, k=3, elite_ratio=0.1):
    elite_size = int(len(population) * elite_ratio)
    elite_indices = np.argsort(fitness_vals)[:elite_size]
    elites = population[elite_indices]

    selected_indices = np.array([random.choices(np.arange(len(fitness_vals)), k=k, weights=1/fitness_vals)[0] for _ in range(len(population) - elite_size)])
    selected = population[selected_indices]

    return np.vstack((elites, selected))

# Cruce basado en ciclos (Cycle Crossover, CX)
def crossover_cycle(parent1, parent2):
    n = len(parent1)
    child = -np.ones(n, dtype=int)
    start = 0
    indices = set()

    while start not in indices:
        indices.add(start)
        start = np.where(parent2 == parent1[start])[0][0]

    for idx in indices:
        child[idx] = parent1[idx]

    for idx in range(n):
        if child[idx] == -1:
            child[idx] = parent2[idx]

    return child

def crossover_population(population, crossover_prob=0.9):
    new_population = []
    for _ in range(len(population) // 2):
        parent1, parent2 = population[random.sample(range(len(population)), 2)]
        if random.random() < crossover_prob:
            child1 = crossover_cycle(parent1, parent2)
            child2 = crossover_cycle(parent2, parent1)
        else:
            child1, child2 = parent1.copy(), parent2.copy()
        new_population.extend([child1, child2])
    return np.array(new_population)

# Mutación adaptativa por intercambio
def mutate_population_adaptive(population, mutation_prob, stagnation_counter, max_stagnation):
    adaptive_prob = mutation_prob + (0.1 * stagnation_counter / max_stagnation)
    adaptive_prob = min(adaptive_prob, 0.5)  # Limitar la mutación adaptativa a un máximo de 50%

    for individual in population:
        if random.random() < adaptive_prob:
            i, j = random.sample(range(len(individual)), 2)
            individual[i], individual[j] = individual[j], individual[i]
    return population

# Búsqueda local (2-opt limitada y paralelizada)
def local_search_2opt_limited(permutation, flow, distances, max_iter=20):
    n = len(permutation)
    best_cost = calculate_cost(permutation, flow, distances)
    for _ in range(max_iter):
        i, j = sorted(random.sample(range(n), 2))
        new_perm = permutation.copy()
        new_perm[i:j+1] = list(reversed(new_perm[i:j+1]))
        new_cost = calculate_cost(new_perm, flow, distances)
        if new_cost < best_cost:
            permutation = new_perm
            best_cost = new_cost
    return permutation

def parallel_local_search(population, flow, distances, max_iter=20, elite_ratio=0.1):
    elite_size = int(len(population) * elite_ratio)
    elite_indices = np.argsort(fitness_pop(population, flow, distances))[:elite_size]
    elite_population = population[elite_indices]

    improved_elites = Parallel(n_jobs=-1)(
        delayed(local_search_2opt_limited)(individual, flow, distances, max_iter)
        for individual in elite_population
    )

    population[elite_indices] = improved_elites
    return population

# Calcular el costo
def calculate_cost(permutation, flow, distances):
    indexed_distances = distances[permutation][:, permutation]
    cost = np.sum(flow * indexed_distances)
    return cost

# Ciclo evolutivo
generations = 200
crossover_prob = 0.9
mutation_prob = 0.3
stagnation_limit = 20  # Número de generaciones sin mejora para detener

best_solution = None
best_fitness = float('inf')
no_improvement_counter = 0

fitness_vals = fitness_pop(population, flow, distances)

for generation in range(generations):
    # Introducir nuevos individuos si hay estancamiento
    if no_improvement_counter >= stagnation_limit // 2:
        new_individuals = np.array([np.random.permutation(np.arange(256)) for _ in range(len(population) // 10)])
        population = np.vstack((population, new_individuals))

    # Búsqueda local paralelizada en la élite
    population = parallel_local_search(population, flow, distances, elite_ratio=0.1)

    # Selección con elitismo
    selected_population = selection_tournament_with_elitism(population, fitness_vals)

    # Cruce
    offspring = crossover_population(selected_population, crossover_prob)

    # Mutación adaptativa
    population = mutate_population_adaptive(offspring, mutation_prob, no_improvement_counter, stagnation_limit)

    # Recalcular fitness
    fitness_vals = fitness_pop(population, flow, distances)

    # Guardar el mejor individuo
    gen_best_fitness = fitness_vals.min()
    if gen_best_fitness < best_fitness:
        best_fitness = gen_best_fitness
        best_solution = population[np.argmin(fitness_vals)]
        no_improvement_counter = 0
    else:
        no_improvement_counter += 1

    print(f"Generación {generation + 1}: Mejor Fitness = {best_fitness}")

    # Criterio de parada por estancamiento
    if no_improvement_counter >= stagnation_limit:
        print("Estancamiento detectado, deteniendo el proceso.")
        break

print("Mejor solución encontrada:", best_solution)
print("Costo asociado:", best_fitness)


Generación 1: Mejor Fitness = 50351294
Generación 2: Mejor Fitness = 49679790
Generación 3: Mejor Fitness = 49679790
Generación 4: Mejor Fitness = 49615980
Generación 5: Mejor Fitness = 49329730
Generación 6: Mejor Fitness = 49329730
Generación 7: Mejor Fitness = 49329730
Generación 8: Mejor Fitness = 49329730
Generación 9: Mejor Fitness = 49329730
Generación 10: Mejor Fitness = 49329730
Generación 11: Mejor Fitness = 49329730
Generación 12: Mejor Fitness = 49329730
Generación 13: Mejor Fitness = 49329730
Generación 14: Mejor Fitness = 49101320
Generación 15: Mejor Fitness = 49101320
Generación 16: Mejor Fitness = 49101320
Generación 17: Mejor Fitness = 49101320
Generación 18: Mejor Fitness = 49101320
Generación 19: Mejor Fitness = 49101320
Generación 20: Mejor Fitness = 49101320
Generación 21: Mejor Fitness = 49101320
Generación 22: Mejor Fitness = 49101320
Generación 23: Mejor Fitness = 49101320
Generación 24: Mejor Fitness = 49101320
Generación 25: Mejor Fitness = 49101320
Generació

In [ ]:
import numpy as np
import random
from joblib import Parallel, delayed

# Lectura de datos desde archivo
data = np.loadtxt('/content/drive/MyDrive/UGR/IC/Practica2/qap.datos/tai256c.dat', skiprows=1)

# Separación en matrices de flujos y distancias
flow = np.int32(data[:256])
distances = np.int32(data[256:])

# Configuración de la semilla para reproducibilidad
seed = 2024
random.seed(seed)
np.random.seed(seed)

# Generación de población inicial con heurísticas mejoradas
def initialize_population(pop_size, n):
    greedy_population = [greedy_initial_solution(flow, distances) for _ in range(pop_size // 4)]
    random_population = [np.random.permutation(np.arange(n)) for _ in range(pop_size - len(greedy_population))]
    return np.array(greedy_population + random_population)

def greedy_initial_solution(flow, distances):
    n = flow.shape[0]
    unassigned = set(range(n))  # Usar un conjunto para un acceso más eficiente
    assigned = []

    # Crear matrices de distancias acumuladas para evitar bucles anidados
    distance_matrix = np.zeros((n, n))

    while unassigned:
        if not assigned:
            facility = random.choice(list(unassigned))
        else:
            last_assigned = assigned[-1]

            # Calcular el costo usando operaciones vectorizadas
            if len(assigned) == 1:
                distance_matrix[last_assigned] = distances[last_assigned]
            else:
                distance_matrix[last_assigned] += distances[last_assigned]

            unassigned_array = np.array(list(unassigned))
            total_costs = np.sum(flow[last_assigned] * distance_matrix[unassigned_array], axis=1)

            min_cost_index = np.argmin(total_costs)
            facility = unassigned_array[min_cost_index]

        assigned.append(facility)
        unassigned.remove(facility)

    return np.array(assigned)



population = initialize_population(2000, 256)

# Función de fitness eficiente usando operaciones matriciales
def fitness_pop(population, flow, distances):
    return np.sum(flow[np.newaxis, :, :] * distances[population[:,:,np.newaxis], population[:, np.newaxis, :]], axis=(1,2))

# Selección por ruleta con diversidad genética
def selection_roulette_diversity(population, fitness_vals, diversity_factor=0.2):
    # Calcular la diversidad como la varianza de cada individuo
    diversity_scores = 1 / (np.var(population, axis=1) + 1e-6)

    # Asegurarse de que las formas coincidan
    if len(fitness_vals) != len(diversity_scores):
        raise ValueError(f"Fitness y diversidad tienen tamaños incompatibles: {len(fitness_vals)} vs {len(diversity_scores)}")

    # Ajustar el fitness combinando fitness_vals y diversity_scores
    adjusted_fitness = fitness_vals * (1 - diversity_factor) + diversity_scores * diversity_factor

    # Normalizar los valores ajustados para formar una distribución de probabilidad
    probabilities = adjusted_fitness / np.sum(adjusted_fitness)

    # Seleccionar individuos con base en la probabilidad
    selected_indices = np.random.choice(np.arange(len(population)), size=len(population), p=probabilities)
    return population[selected_indices]

# Recalcular fitness después de cada cambio en la población
fitness_vals = fitness_pop(population, flow, distances)

# Asegurar que los tamaños sean compatibles
selected_population = selection_roulette_diversity(population, fitness_vals)



# Cruce basado en ciclos (Cycle Crossover, CX)
def crossover_cycle(parent1, parent2):
    n = len(parent1)
    child = -np.ones(n, dtype=int)
    start = 0
    indices = set()

    while start not in indices:
        indices.add(start)
        start = np.where(parent2 == parent1[start])[0][0]

    for idx in indices:
        child[idx] = parent1[idx]

    for idx in range(n):
        if child[idx] == -1:
            child[idx] = parent2[idx]

    return child

def crossover_population(population, crossover_prob=0.9):
    new_population = []
    for _ in range(len(population) // 2):
        parent1, parent2 = population[random.sample(range(len(population)), 2)]
        if random.random() < crossover_prob:
            child1 = crossover_cycle(parent1, parent2)
            child2 = crossover_cycle(parent2, parent1)
        else:
            child1, child2 = parent1.copy(), parent2.copy()
        new_population.extend([child1, child2])
    return np.array(new_population)

# Mutación adaptativa por intercambio
def mutate_population_adaptive(population, mutation_prob, stagnation_counter, max_stagnation):
    adaptive_prob = mutation_prob + (0.1 * stagnation_counter / max_stagnation)
    adaptive_prob = min(adaptive_prob, 0.5)  # Limitar la mutación adaptativa a un máximo de 50%

    for individual in population:
        if random.random() < adaptive_prob:
            i, j = random.sample(range(len(individual)), 2)
            individual[i], individual[j] = individual[j], individual[i]
    return population

# Búsqueda local con Enfriamiento Simulado
def simulated_annealing_local_search(permutation, flow, distances, max_iter=100, initial_temp=1000, cooling_rate=0.95):
    current_solution = permutation.copy()
    current_cost = calculate_cost(current_solution, flow, distances)
    best_solution = current_solution.copy()
    best_cost = current_cost

    temperature = initial_temp
    for _ in range(max_iter):
        i, j = sorted(random.sample(range(len(permutation)), 2))
        new_solution = current_solution.copy()
        new_solution[i:j+1] = list(reversed(new_solution[i:j+1]))
        new_cost = calculate_cost(new_solution, flow, distances)

        if new_cost < current_cost or random.random() < np.exp((current_cost - new_cost) / temperature):
            current_solution = new_solution
            current_cost = new_cost

        if current_cost < best_cost:
            best_solution = current_solution
            best_cost = current_cost

        temperature *= cooling_rate

    return best_solution

# Aplicar búsqueda local en la élite
def apply_local_search(population, flow, distances, elite_ratio=0.1):
    elite_size = int(len(population) * elite_ratio)
    elite_indices = np.argsort(fitness_pop(population, flow, distances))[:elite_size]
    for idx in elite_indices:
        population[idx] = simulated_annealing_local_search(population[idx], flow, distances)
    return population

# Calcular el costo
def calculate_cost(permutation, flow, distances):
    indexed_distances = distances[permutation][:, permutation]
    cost = np.sum(flow * indexed_distances)
    return cost

# Ciclo evolutivo
generations = 300
crossover_prob = 0.9
mutation_prob = 0.3
stagnation_limit = 20  # Número de generaciones sin mejora para detener

best_solution = None
best_fitness = float('inf')
no_improvement_counter = 0

fitness_vals = fitness_pop(population, flow, distances)

for generation in range(generations):
    # Introducir nuevos individuos si hay estancamiento
    if no_improvement_counter >= stagnation_limit // 2:
        new_individuals = np.array([np.random.permutation(np.arange(256)) for _ in range(len(population) // 10)])
        population = np.vstack((population, new_individuals))

        # Recalcular fitness después de modificar la población
        fitness_vals = fitness_pop(population, flow, distances)

    # Búsqueda local con enfriamiento simulado
    population = apply_local_search(population, flow, distances, elite_ratio=0.1)

    # Recalcular fitness después de la búsqueda local
    fitness_vals = fitness_pop(population, flow, distances)

    # Selección con diversidad genética
    selected_population = selection_roulette_diversity(population, fitness_vals)

    # Cruce
    offspring = crossover_population(selected_population, crossover_prob)

    # Mutación adaptativa
    population = mutate_population_adaptive(offspring, mutation_prob, no_improvement_counter, stagnation_limit)

    # Recalcular fitness después de la mutación
    fitness_vals = fitness_pop(population, flow, distances)

    # Guardar el mejor individuo
    gen_best_fitness = fitness_vals.min()
    if gen_best_fitness < best_fitness:
        best_fitness = gen_best_fitness
        best_solution = population[np.argmin(fitness_vals)]
        no_improvement_counter = 0
    else:
        no_improvement_counter += 1

    print(f"Generación {generation + 1}: Mejor Fitness = {best_fitness}")

    # Criterio de parada por estancamiento
    if no_improvement_counter >= stagnation_limit:
        print("Estancamiento detectado, deteniendo el proceso.")
        break

print("Mejor solución encontrada:", best_solution)
print("Costo asociado:", best_fitness)


Generación 1: Mejor Fitness = 49722166
Generación 2: Mejor Fitness = 49722166
Generación 3: Mejor Fitness = 49537942
Generación 4: Mejor Fitness = 49300616
Generación 5: Mejor Fitness = 49300616
Generación 6: Mejor Fitness = 49300616
Generación 7: Mejor Fitness = 49300616
Generación 8: Mejor Fitness = 49300616
Generación 9: Mejor Fitness = 49300616
Generación 10: Mejor Fitness = 49300616
Generación 11: Mejor Fitness = 49300616
Generación 12: Mejor Fitness = 49300616
Generación 13: Mejor Fitness = 49300616
Generación 14: Mejor Fitness = 49300616
Generación 15: Mejor Fitness = 49300616
Generación 16: Mejor Fitness = 49300616
Generación 17: Mejor Fitness = 49300616
Generación 18: Mejor Fitness = 49300616
Generación 19: Mejor Fitness = 49300616
Generación 20: Mejor Fitness = 49300616
Generación 21: Mejor Fitness = 49300616
Generación 22: Mejor Fitness = 49300616
Generación 23: Mejor Fitness = 49300616
Generación 24: Mejor Fitness = 49204732
Generación 25: Mejor Fitness = 49204732
Generació